In [1]:
import numpy as np
import lightgbm as lgb
from wideboost.wrappers import wlgb

import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

(ds_train, ds_test), ds_info = tfds.load(
    'fashion_mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

for i in ds_train.batch(60000):
    a = i
    break
    
for i in ds_test.batch(60000):
    b = i
    break

In [2]:
xtrain = a[0].numpy().reshape([-1,28*28])
ytrain = a[1].numpy()

xtest = b[0].numpy().reshape([-1,28*28])
ytest = b[1].numpy()

#dtrain = xgb.DMatrix(xtrain,label=ytrain)
#dtest = xgb.DMatrix(xtest,label=ytest)

train_data = lgb.Dataset(xtrain, label=ytrain)
test_data = lgb.Dataset(xtest, label=ytest)

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval

best_val = 1.0

def objective(param):
    global best_val
    #watchlist = [(dtrain,'train'),(dtest,'test')]
    ed1_results = dict()
    print(param)
    param['num_leaves'] = round(param['num_leaves']+1)
    param['min_data_in_leaf'] = round(param['min_data_in_leaf'])
    wbst = lgb.train(param,
                      train_data,
                      num_boost_round=20,
                      valid_sets=test_data,
                      evals_result=ed1_results)
    output = min(ed1_results['valid_0']['multi_error'])
    
    if output < best_val:
        print("NEW BEST VALUE!")
        best_val = output
    
    return {'loss': output, 'status': STATUS_OK }

spc = {
    'objective': hp.choice('objective',['multiclass']),
    'metric':hp.choice('metric',['multi_error']),
    'num_class':hp.choice('num_class',[10]),
    'learning_rate': hp.loguniform('learning_rate', -7, 0),
    'num_leaves' : hp.qloguniform('num_leaves', 0, 7, 1),
    'feature_fraction': hp.uniform('feature_fraction', 0.5, 1),
    'bagging_fraction': hp.uniform('bagging_fraction', 0.5, 1),
    'min_data_in_leaf': hp.qloguniform('min_data_in_leaf', 0, 6, 1),
    'min_sum_hessian_in_leaf': hp.loguniform('min_sum_hessian_in_leaf', -16, 5),
    'lambda_l1': hp.choice('lambda_l1', [0, hp.loguniform('lambda_l1_positive', -16, 2)]),
    'lambda_l2': hp.choice('lambda_l2', [0, hp.loguniform('lambda_l2_positive', -16, 2)])
}


best = fmin(objective,
    space=spc,
    algo=tpe.suggest,
    max_evals=100)

{'bagging_fraction': 0.7251796751234882, 'feature_fraction': 0.5933005337725001, 'lambda_l1': 1.3372979090365175e-06, 'lambda_l2': 0, 'learning_rate': 0.533315059142865, 'metric': 'multi_error', 'min_data_in_leaf': 1.0, 'min_sum_hessian_in_leaf': 0.00026298877164697437, 'num_class': 10, 'num_leaves': 33.0, 'objective': 'multiclass'}
[1]	valid_0's multi_error: 0.2029                      
[2]	valid_0's multi_error: 0.1716                      
[3]	valid_0's multi_error: 0.1623                      
[4]	valid_0's multi_error: 0.1593                      
[5]	valid_0's multi_error: 0.155                       
[6]	valid_0's multi_error: 0.152                       
[7]	valid_0's multi_error: 0.1489                      
[8]	valid_0's multi_error: 0.1466                      
[9]	valid_0's multi_error: 0.1444                      
[10]	valid_0's multi_error: 0.1431                     
[11]	valid_0's multi_error: 0.1416                     
[12]	valid_0's multi_error: 0.1377               

[14]	valid_0's multi_error: 0.2339                                     
[15]	valid_0's multi_error: 0.23                                       
[16]	valid_0's multi_error: 0.2291                                     
[17]	valid_0's multi_error: 0.2261                                     
[18]	valid_0's multi_error: 0.2231                                     
[19]	valid_0's multi_error: 0.2212                                     
[20]	valid_0's multi_error: 0.2202                                     
{'bagging_fraction': 0.5038484260910951, 'feature_fraction': 0.7515169606255043, 'lambda_l1': 7.171421157866604e-05, 'lambda_l2': 0.01241835314152486, 'learning_rate': 0.050035092919077796, 'metric': 'multi_error', 'min_data_in_leaf': 230.0, 'min_sum_hessian_in_leaf': 0.017423771764820038, 'num_class': 10, 'num_leaves': 229.0, 'objective': 'multiclass'}
[1]	valid_0's multi_error: 0.1974                                      
[2]	valid_0's multi_error: 0.1817                                   

[4]	valid_0's multi_error: 0.2957                                    
[5]	valid_0's multi_error: 0.2862                                    
[6]	valid_0's multi_error: 0.2864                                    
[7]	valid_0's multi_error: 0.2845                                    
[8]	valid_0's multi_error: 0.2741                                    
[9]	valid_0's multi_error: 0.2719                                    
[10]	valid_0's multi_error: 0.2729                                   
[11]	valid_0's multi_error: 0.2738                                   
[12]	valid_0's multi_error: 0.2737                                   
[13]	valid_0's multi_error: 0.2741                                   
[14]	valid_0's multi_error: 0.2736                                   
[15]	valid_0's multi_error: 0.2712                                   
[16]	valid_0's multi_error: 0.2702                                   
[17]	valid_0's multi_error: 0.2715                                   
[18]	valid_0's multi

[19]	valid_0's multi_error: 0.1224                                      
[20]	valid_0's multi_error: 0.1213                                      
{'bagging_fraction': 0.5977927218479516, 'feature_fraction': 0.6262331116806609, 'lambda_l1': 0, 'lambda_l2': 1.1653516248434512e-07, 'learning_rate': 0.06182945427091742, 'metric': 'multi_error', 'min_data_in_leaf': 2.0, 'min_sum_hessian_in_leaf': 0.7040178421392984, 'num_class': 10, 'num_leaves': 2.0, 'objective': 'multiclass'}
[1]	valid_0's multi_error: 0.3646                                       
[2]	valid_0's multi_error: 0.3196                                       
[3]	valid_0's multi_error: 0.3122                                       
[4]	valid_0's multi_error: 0.299                                        
[5]	valid_0's multi_error: 0.2936                                       
[6]	valid_0's multi_error: 0.2872                                       
[7]	valid_0's multi_error: 0.2852                                       
[8]	valid_0

[9]	valid_0's multi_error: 0.222                                        
[10]	valid_0's multi_error: 0.2211                                      
[11]	valid_0's multi_error: 0.2221                                      
[12]	valid_0's multi_error: 0.2236                                      
[13]	valid_0's multi_error: 0.2237                                      
[14]	valid_0's multi_error: 0.2213                                      
[15]	valid_0's multi_error: 0.221                                       
[16]	valid_0's multi_error: 0.221                                       
[17]	valid_0's multi_error: 0.22                                        
[18]	valid_0's multi_error: 0.2198                                      
[19]	valid_0's multi_error: 0.2197                                      
[20]	valid_0's multi_error: 0.2195                                      
{'bagging_fraction': 0.5299560443148827, 'feature_fraction': 0.8133464633686073, 'lambda_l1': 0, 'lambda_l2': 0, 'learning_r

...

[8]	valid_0's multi_error: 0.2083                                      
[9]	valid_0's multi_error: 0.2124                                      
[10]	valid_0's multi_error: 0.2124                                     
[11]	valid_0's multi_error: 0.2154                                     
[12]	valid_0's multi_error: 0.2213                                     
[13]	valid_0's multi_error: 0.2258                                     
[14]	valid_0's multi_error: 0.2272                                     
[15]	valid_0's multi_error: 0.2297                                     
[16]	valid_0's multi_error: 0.2346                                     
[17]	valid_0's multi_error: 0.2466                                     
[18]	valid_0's multi_error: 0.2478                                     
[19]	valid_0's multi_error: 0.2516                                     
[20]	valid_0's multi_error: 0.2413                                     
{'bagging_fraction': 0.75445433935413, 'feature_fraction': 0.781

[1]	valid_0's multi_error: 0.1757                                      
[2]	valid_0's multi_error: 0.1577                                      
[3]	valid_0's multi_error: 0.1476                                      
[4]	valid_0's multi_error: 0.1444                                      
[5]	valid_0's multi_error: 0.14                                        
[6]	valid_0's multi_error: 0.1361                                      
[7]	valid_0's multi_error: 0.1357                                      
[8]	valid_0's multi_error: 0.1324                                      
[9]	valid_0's multi_error: 0.1315                                      
[10]	valid_0's multi_error: 0.1322                                     
[11]	valid_0's multi_error: 0.1305                                     
[12]	valid_0's multi_error: 0.1288                                     
[13]	valid_0's multi_error: 0.1257                                     
[14]	valid_0's multi_error: 0.1239                              

[16]	valid_0's multi_error: 0.1497                                     
[17]	valid_0's multi_error: 0.1495                                     
[18]	valid_0's multi_error: 0.1492                                     
[19]	valid_0's multi_error: 0.149                                      
[20]	valid_0's multi_error: 0.1484                                     
{'bagging_fraction': 0.5187318390244715, 'feature_fraction': 0.5025958926243889, 'lambda_l1': 1.890287060898057e-06, 'lambda_l2': 0, 'learning_rate': 0.5983231390863334, 'metric': 'multi_error', 'min_data_in_leaf': 112.0, 'min_sum_hessian_in_leaf': 1.7745616678010998e-07, 'num_class': 10, 'num_leaves': 54.0, 'objective': 'multiclass'}
[1]	valid_0's multi_error: 0.1868                                      
[2]	valid_0's multi_error: 0.1676                                      
[3]	valid_0's multi_error: 0.1527                                      
[4]	valid_0's multi_error: 0.1443                                      
[5]	valid_0's m

[6]	valid_0's multi_error: 0.1313                                      
[7]	valid_0's multi_error: 0.1278                                      
[8]	valid_0's multi_error: 0.1284                                      
[9]	valid_0's multi_error: 0.126                                       
[10]	valid_0's multi_error: 0.1238                                     
[11]	valid_0's multi_error: 0.1228                                     
[12]	valid_0's multi_error: 0.1224                                     
[13]	valid_0's multi_error: 0.1208                                     
[14]	valid_0's multi_error: 0.1205                                     
[15]	valid_0's multi_error: 0.1199                                     
[16]	valid_0's multi_error: 0.118                                      
[17]	valid_0's multi_error: 0.1171                                     
[18]	valid_0's multi_error: 0.1164                                     
[19]	valid_0's multi_error: 0.1161                              

{'bagging_fraction': 0.5891080492904942, 'feature_fraction': 0.680264069561943, 'lambda_l1': 1.2304093624300756, 'lambda_l2': 0, 'learning_rate': 0.3198422946320555, 'metric': 'multi_error', 'min_data_in_leaf': 3.0, 'min_sum_hessian_in_leaf': 0.00023807197603505613, 'num_class': 10, 'num_leaves': 32.0, 'objective': 'multiclass'}
[1]	valid_0's multi_error: 0.2028                                      
[2]	valid_0's multi_error: 0.1724                                      
[3]	valid_0's multi_error: 0.1619                                      
[4]	valid_0's multi_error: 0.1529                                      
[5]	valid_0's multi_error: 0.1497                                      
[6]	valid_0's multi_error: 0.1468                                      
[7]	valid_0's multi_error: 0.1426                                      
[8]	valid_0's multi_error: 0.1422                                      
[9]	valid_0's multi_error: 0.1407                                      
[10]	valid_0's multi_

In [4]:
print(best_val)
print(space_eval(spc, best))

0.112
{'bagging_fraction': 0.6169757023287703, 'feature_fraction': 0.8932740248835392, 'lambda_l1': 0.09422621802317309, 'lambda_l2': 0, 'learning_rate': 0.832326366852246, 'metric': 'multi_error', 'min_data_in_leaf': 51.0, 'min_sum_hessian_in_leaf': 8.927831603309363e-07, 'num_class': 10, 'num_leaves': 682.0, 'objective': 'multiclass'}
